### Generate a Complete Database 

- date : 24/05/2024
- New features : generate a complete database using "OrdalieTech/Solon-embeddings-large-0.1" embedding model (config file have been improved)
- Expected improvement : improving the retrieval capabilities with a much strong embedding model.

In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')), '..', 'llm-open-data-insee/src')))

### Building New Complete Dataset based on config files 

In [ ]:
from db_building import  build_database_from_csv
#db = build_database_from_csv('/home/onyxia/work/llm-open-data-insee/data_complete.csv')
db.similarity_search("Quels sont les chiffres du chômages en 2023")

### Loading Dataset based on config files

In [2]:
from db_building import reload_database_from_local_dir
db = reload_database_from_local_dir(persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")

2024-06-14 15:22:51,587 - INFO - Load pretrained SentenceTransformer: OrdalieTech/Solon-embeddings-large-0.1
/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-14 15:22:55,471 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-06-14 15:22:55,596 - INFO - Collection insee_data is not created.
2024-06-14 15:22:55,598 - INFO - The database (collection insee_data) has been reloaded from directory /home/onyxia/work/llm-open-data-insee/data/chroma_db


In [3]:
len(db.get()["ids"])

101833

In [4]:
result = db.similarity_search("Quels résultats au BAC les étudiants de classes préparatoires ont ils généralement?", k = 5 )
print(result)

2024-06-14 15:23:08,695 - INFO - Start multi-process pool on devices: cuda:0


: 

In [ ]:
from config import RAG_PROMPT_TEMPLATE, EMB_MODEL_NAME, MODEL_NAME
from model_building import build_llm_model
from chain_building.build_chain import (
    load_retriever,
    build_chain
    )

from langchain_core.prompts import PromptTemplate
import chainlit as cl

prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)
#retriever = load_retriever(emb_model_name=EMB_MODEL_NAME,
                        #persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")
retriever = db.as_retriever(search_type="mmr", search_kwargs={"score_threshold": 0.5, "k": 10})
llm = build_llm_model(model_name=MODEL_NAME,
                        quantization_config=True,
                        config=True,
                        token=os.environ["HF_TOKEN"])
chain = build_chain(retriever, prompt, llm)


In [12]:
question = "Quels résultats au BAC les étudiants de classes préparatoires ont ils généralement?" #str(input('write your question : '))
result = await chain.ainvoke({"question": question }) 

print('question posée : ', result["question"]["question"])
print('Documents récupérés : \n' , "\n".join([doc.page_content for doc in result["context"]]))
print("Réponse : : \n", result["answer"])

question posée :  Chiffre de l'inflation
Documents récupérés : 
 . Les questions trimestrielles complètent l’enquête mensuelle publiée le 23 janvier 2019, notamment sur la demande et les facteurs de production. Le solde d’opinion correspond à l’écart entre le pourcentage de réponses «en hausse» et le pourcentage de réponses «en baisse». Des informations complémentaires (méthodologie simplifiée et détaillée,nomenclatures, etc.) sont disponibles par l’onglet «Documentation» de la présente page.  Prochaine publication: le 24 avril 2019 à 8h45
.) sont disponibles par l’onglet «Documentation» de la page web de cette publication Les réponses à cette enquête ont été collectées entre le 27maiet le 21juin2021. Prochaine publicationle 22juillet2021 à 8h45.
. Les réponses à cette enquête ont été collectées entre le 28 décembre 2022 et le 23 janvier 2023.  Des informations complémentaires (méthodologie simplifiée et détaillée,nomenclatures, etc.) sont disponibles par l’onglet «Documentation» de pa